# Training and evaluation on 3DPW dataset for SoMoF challenge
## Quick dataset setup
**Create data directories:**

In [ ]:
!mkdir ./data
!mkdir ./data/3dpw
#!mkdir ./data/amass

**Download files:**

- Download `sequenceFiles.zip` from [https://virtualhumans.mpi-inf.mpg.de/3DPW/license.html](https://virtualhumans.mpi-inf.mpg.de/3DPW/license.html) and place it in `./data/3dpw` folder


- Download SoMoF challenge dataset (`somof_data_3dpw.zip`) from [https://virtualhumans.mpi-inf.mpg.de/3DPW/licensesmof.html](https://virtualhumans.mpi-inf.mpg.de/3DPW/licensesmof.html) and place the zip in `./data` folder


- Download `CMU`(*CMU.tar.bz2*), `BMLmovi` (*BMLmovi.tar.bz2*), `BioMotionLab_NTroje` (*BMLrub.tar.bz2*) from [https://amass.is.tue.mpg.de/](https://amass.is.tue.mpg.de/) and place `.tar.bz2` files in `./data/amass`folder



**Extract files:**

In [ ]:
!unzip ./data/somof_data_3dpw.zip -d ./data
!unzip ./data/3dpw/sequenceFiles.zip -d ./data/3dpw/

!tar -xf ./data/amass/CMU.tar.bz2 -C ./data/amass
!tar -xf ./data/amass/BMLmovi.tar.bz2 -C ./data/amass
!tar -xf ./data/amass/BMLrub.tar.bz2 -C ./data/amass

<br/>

# Train the model on 3DPW and AMASS

In [ ]:
!python train.py --device cuda --use_dct --dct_n 30 --use_full_augmentation --use_ctx_loss --out_model_name mpfsir_full

In [ ]:
!python eval.py --device cpu --dataset 3dpw --dct_n 30 --out_model_name mpfsir_full

<br/><br/>

# Training and evaluation on CMU & MuPoTS dataset
## Quick dataset setup

In [ ]:
!mkdir support
%cd support
!git clone https://github.com/jiashunwang/MRT.git
%cd ..

- Download CMU dataset [http://mocap.cs.cmu.edu/allasfamc.zip](http://mocap.cs.cmu.edu/allasfamc.zip) and place it in `./support/MRT/mocap` folder
- Download MuPoTS dataset [http://gvv.mpi-inf.mpg.de/projects/SingleShotMultiPerson/MultiPersonTestSet.zip](http://gvv.mpi-inf.mpg.de/projects/SingleShotMultiPerson/MultiPersonTestSet.zip) and place it in `./support/MRT/mupots3d` folder as `data` folder 

**or run the two following commands:**

In [ ]:
!wget -O ./support/MRT/mocap/allasfamc.zip http://mocap.cs.cmu.edu/allasfamc.zip
!wget -O ./support/MRT/mupots3d/MultiPersonTestSet.zip http://gvv.mpi-inf.mpg.de/projects/SingleShotMultiPerson/MultiPersonTestSet.zip

In [ ]:
!unzip ./support/MRT/mocap/allasfamc.zip -d ./support/MRT/mocap/
!unzip ./support/MRT/mupots3d/MultiPersonTestSet.zip -d ./support/MRT/mupots3d
!mv ./support/MRT/mupots3d/MultiPersonTestSet ./support/MRT/mupots3d/data

**Create datasets:** (<span style="color:red;">**Beware:**</span> `mix_mocap.py` uses random samples from datasets, thus the results may be very different depending on the generated datasets)

In [ ]:
%cd ./support/MRT/mocap
!python preprocess_mocap.py
!python mix_mocap.py
%cd ..

%cd mupots3d
!python preprocess_mupots.py
%cd ../../..

**Copy dataset files in `data` folder:**

In [ ]:
!mkdir ./data/cmu_mupots
!cp ./support/MRT/mocap/train_3_120_mocap.npy ./data/cmu_mupots/train_3_120_mocap.npy
!cp ./support/MRT/mocap/test_3_120_mocap.npy ./data/cmu_mupots/test_3_120_mocap.npy
!cp ./support/MRT/mupots3d/mupots_120_3persons.npy ./data/cmu_mupots/mupots_120_3persons.npy

**Train and evaluate the model:**

In [ ]:
!python train_cmu.py --device cuda --use_dct --dct_n 40 --use_full_augmentation --use_ctx_loss --out_model_name cmu_mupots_mpfsir


In [ ]:
!python eval.py --device cpu --dataset cmu --dct_n 40 --out_model_name cmu_mupots_mpfsir

<br/><br/>

## Ablation study

**Ablation: model baseline:**

In [ ]:
!python train.py --device cuda --ablation --independent_ctx --out_model_name mpfsir_ablation_independent

In [ ]:
!python eval.py --device cpu --ablation --independent_ctx --dataset 3dpw --out_model_name mpfsir_ablation_independent

<br/><br/>**Ablation: model with Temporal&Spacial CTX:**

In [ ]:
!python train.py --device cuda --ablation --out_model_name mpfsir_ablation_normal

In [ ]:
!python eval.py --device cpu --ablation --dataset 3dpw --out_model_name mpfsir_ablation_normal

<br/><br/>**Ablation: model with DCT:**

In [ ]:
!python train.py --device cuda --ablation --use_dct --dct_n 30 --out_model_name mpfsir_ablation_normal_dct

In [ ]:
!python eval.py --device cpu --ablation --use_dct --dct_n 30 --dataset 3dpw --out_model_name mpfsir_ablation_normal_dct

<br/><br/>**Ablation: model with data augmentation:**

In [ ]:
!python train.py --device cuda --ablation --use_dct --dct_n 30 --use_full_augmentation --out_model_name mpfsir_ablation_normal_dct_augmented

In [ ]:
!python eval.py --device cpu --ablation --use_dct --dct_n 30 --dataset 3dpw --out_model_name mpfsir_ablation_normal_dct_augmented

<br/><br/>**Ablation: model with Social-interaction loss:**

In [ ]:
!python train.py --device cuda --ablation --use_dct --dct_n 30 --use_full_augmentation --use_ctx_loss --out_model_name mpfsir_ablation_full

In [ ]:
!python eval.py --device cpu --ablation --use_dct --dct_n 30 --dataset 3dpw --out_model_name mpfsir_ablation_full